In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from lxml import html
from htmldate import find_date
from tkinter import *
import webbrowser

sources = ("https://tokenpost.com", "https://www.coindesk.com", "https://www.supplychaindive.com", 
           "https://www.newsbtc.com", "https://www.forbes.com")
urls = []
for home in sources:
    avoid = ['All Blockchain 101 Guides', '02How Does Blockchain Technology Work?', '03What Can a Blockchain Do?', 
            '01What is Blockchain Technology?', '05Why Use a Blockchain?', '06How Could Blockchain Technology Change Finance?']
    result = requests.get(home, headers={'User-Agent': 'Chrome/63.0.3239.132'})
    src = result.content
    soup = BeautifulSoup(src, 'lxml')
    a_tags = soup.find_all('a')
    for a_tag in a_tags:
        if "blockchain" in a_tag.text.lower():
            if a_tag.text not in avoid:
                if "https" in a_tag.attrs['href']:
                    link = a_tag.attrs['href']
                    date = find_date(link, outputformat='%d %B %Y')
                    title = a_tag.text.strip(), date, link
                    urls.append(title)
                else:
                    ext = a_tag.attrs['href']
                    link = home + ext
                    date = find_date(link, outputformat='%d %B %Y')
                    title = a_tag.text.strip(), date, link
                    urls.append(title)

home_alone = "https://cointelegraph.com"
result = requests.get(home_alone, headers={'User-Agent': 'Chrome/63.0.3239.132'})
src = result.content
soup = BeautifulSoup(src, 'lxml')
a_tags = soup.find_all('a')
for a_tag in a_tags:
    if "blockchain" in a_tag.text.lower():
        if "https" in a_tag.attrs['href']:
            link = a_tag.attrs['href']
            fetch = requests.get(link, headers={'User-Agent': 'Chrome/63.0.3239.132'})
            source = fetch.content
            target = html.fromstring(source)
            date = find_date(target, outputformat='%d %B %Y')
            title = a_tag.text.strip(), date, link
            urls.append(title)
        else:
            ext = a_tag.attrs['href']
            link = home + ext
            fetch = requests.get(link, headers={'User-Agent': 'Chrome/63.0.3239.132'})
            source = fetch.content
            target = html.fromstring(source)
            date = find_date(target, outputformat='%d %B %Y')
            title = a_tag.text.strip(), date, link
            urls.append(title)

def removeDuplicates(lst): 
      
    return list(set([i for i in lst]))


articles = removeDuplicates(urls)                
articles.sort(key=lambda date: datetime.strptime(date[1], "%d %B %Y"), reverse = True)
root = Tk()
gui_header = Label(root, text="Latest Blockchain Articles", fg="black", font=40).pack()
gui_space = Label(root, text="                            ", font = 30).pack()
labels=[]
eval_link = lambda x: (lambda p: webbrowser.open(x))
for article in articles:
    label = Label(root, text=f"{article[0]}, {article[1]}", fg="blue", font=15, cursor="hand2")
    label.pack()
    label.bind("<Button-1>", eval_link(article[2]))
    labels.append(label)
    
root.mainloop()